# Gerson Andrade
# Tarea: Analisis exploratorio con Topic Modeling
## 1. Extraer las noticias de una semana especifica en los medios de la region de Los Lagos


In [450]:
import elasticsearch
from datetime import datetime

In [451]:
elasticsearch.__version__

(7, 17, 0)

#### Conexión a Sophia

In [452]:
country="chile"
from_="2020-01-01"
to_="2020-01-08"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

simple_keyword=False

In [453]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [454]:
comunas= ["Puerto Montt", "Calbuco", "Cochamó", "Fresia", "Frutillar", "Los Muermos", "Llanquihue", "Maullín", "Puerto Varas", "Castro", "Ancud", "Chonchi", "Curaco de Vélez", "Dalcahue", "Puqueldón", "Queilén", "Quellón", "Quemchi", "Quinchao", "Osorno", "Puerto Octay", "Purranque", "Puyehue", "Río Negro", "San Juan de la Costa", "San Pablo", "Chaitén", "Futaleufú", "Hualaihué", "Palena"]


In [455]:
es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [456]:
#NO MODIFICAR
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [457]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 1220 noticias encontradas...


In [458]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}


    #Se agregan comunas a la fila
    for comuna in comunas:
        if comuna in text:
            new_row[comuna] = text.count(comuna)
        else:
            new_row[comuna] = 0

    df = df.append(new_row, ignore_index=True)
    

dfComunas = df.drop_duplicates(subset='url', keep='first')

C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\1403750888.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\1403750888.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\1403750888.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\1403750888.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Gerso\A

In [486]:
dfComunas

,id_news,country,media_outlet,url,title,text,date,Ancud,Calbuco,Castro,...,Puqueldón,Purranque,Puyehue,Queilén,Quellón,Quemchi,Quinchao,Río Negro,San Juan de la Costa,San Pablo
0,21855800.0,chile,radiosago,https://www.radiosago.cl/se-aprueban-recursos-para-construccion-de-nuevas-veredas-en-cancura/,Se aprueban recursos para construcción de nuevas veredas en Cancura,"Los charcos de barro y socavones en invierno y las grandes cantidades de polvo en verano, son algunos de los problemas que afectan el desplazamiento de los vecinos de la localidad de Cancura en la comuna de Osorno, debido a la falta de veredas en las principales vías de tránsito del sector. Se trata de un problema que arrastra hace años, pero para el que recientemente se adjudicaron recursos que permitirán dar lugar a una solución definitiva. Lo anterior, luego de que la Subsecretaría de Desarrollo Regional y Administrativo (Subdere), a través de su Programa de Mejoramiento Urbano (PMU), decidiera destinar $ 38 millones para estos fines. De acuerdo al proyecto presentado por el municipio, se requiere la construcción de veredas en las principales vías de tránsito del sector, las cuales deben considerar bajadas de vehículos y accesos universales en aquellos lugares que presentan una mayor afluencia de personas, sobre todo en época estival. De este modo, los 1.000 vecinos y vecinas qu...",2020-01-07,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21855878.0,chile,radiosago,https://www.radiosago.cl/concejo-municipal-aprobo-20-millones-de-pesos-para-desarrollo-de-programa-turismo-familiar-2020/,Concejo municipal aprobó $20 millones de pesos para desarrollo de programa Turismo Familiar 2020,"Por unanimidad el Concejo Municipal de Osorno resolvió aprobar 20 millones de pesos para desarrollar el programa municipal “Turismo Familiar 2020”, dineros que serán remitidos a la Unión Comunal de Juntas de Vecinos, quienes tendrán la misión de recibir las solicitudes de todas las organizaciones que deseen postular a esta iniciativa vacacional durante la temporada estival. La iniciativa municipal busca brindar espacios de recreación y esparcimiento para las familias de la comuna que, por diversos motivos, no pueden salir de Osorno en esta época. La idea es que tanto niños, adultos y adultos mayores puedan disfrutar tanto en zonas campestres como en balnearios de la zona. La presidenta de la Unión Comunal de Juntas de Vecinos de Osorno, Eliana Catrilef, agradeció la voluntad de la casa edilicia local “al brindar un aporte que permite que las familias puedan optar a planes de veraneo que, de forma particular, les sería difícil o imposible acceder, permitiendo con esto que tanto los ...",2020-01-03,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21855983.0,chile,radiosago,https://www.radiosago.cl/el-intendente-de-los-lagos-aseguro-que-en-los-proximos-tres-anos-se-entregaran-tres-mil-soluciones-habitacionales/,El Intendente de Los Lagos aseguró que en los próximos tres años se entregarán tres mil soluciones habitacionales,"Es conocido el déficit habitacional que existe en la ciudad de Osorno, y se ha evidenciado en demasía durante el último año con una serie de manifestaciones en torno a esta situación, sin ir más lejos las tomas de campamentos en diferentes zonas de la comuna han demostrado que esto ocurre. El jefe regional Harry Jurgensen, manifestó que se está analizando la creación de un plan especial para entregar tres mil soluciones habitacionales para los próximos tres años en Osorno. En este sentido se espera la visita del Ministro de Vivienda y Urbanismo el 2020 para reafirmar este anuncio. Por su parte, el diputado por Osorno, Fidel Espinoza, indicó que debe haber una suerte de cautela en estos anuncios, ya que se han hecho en el pasado y no se han cumplido a cabalidad. Asimismo dijo que estará de cerca fiscalizando este compromiso, ya que presidirá la comisión de vivienda en la Cámara de Diputados a partir de marzo de 2020. Por parte del Gobierno, el inten

In [460]:
nombre_archivo=keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
dfComunas.to_csv("./"+nombre_archivo)

El archivo se llama:
_chile_2020-01-01_2020-01-08.csv


In [461]:
# leer el archivo CSV
archivo = "./_chile_2020-01-01_2020-01-08.csv"
corpus = pd.read_csv(archivo)
corpus

,Unnamed: 0,id_news,country,media_outlet,url,title,text,date,Ancud,Calbuco,...,Puqueldón,Purranque,Puyehue,Queilén,Quellón,Quemchi,Quinchao,Río Negro,San Juan de la Costa,San Pablo
0,0,21855800.0,chile,radiosago,https://www.radiosago.cl/se-aprueban-recursos-para-construccion-de-nuevas-veredas-en-cancura/,Se aprueban recursos para construcción de nuevas veredas en Cancura,"Los charcos de barro y socavones en invierno y las grandes cantidades de polvo en verano, son algunos de los problemas que afectan el desplazamiento de los vecinos de la localidad de Cancura en la comuna de Osorno, debido a la falta de veredas en las principales vías de tránsito del sector. Se trata de un problema que arrastra hace años, pero para el que recientemente se adjudicaron recursos que permitirán dar lugar a una solución definitiva. Lo anterior, luego de que la Subsecretaría de Desarrollo Regional y Administrativo (Subdere), a través de su Programa de Mejoramiento Urbano (PMU), decidiera destinar $ 38 millones para estos fines. De acuerdo al proyecto presentado por el municipio, se requiere la construcción de veredas en las principales vías de tránsito del sector, las cuales deben considerar bajadas de vehículos y accesos universales en aquellos lugares que presentan una mayor afluencia de personas, sobre todo en época estival. De este modo, los 1.000 vecinos y vecinas qu...",2020-01-07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,21855878.0,chile,radiosago,https://www.radiosago.cl/concejo-municipal-aprobo-20-millones-de-pesos-para-desarrollo-de-programa-turismo-familiar-2020/,Concejo municipal aprobó $20 millones de pesos para desarrollo de programa Turismo Familiar 2020,"Por unanimidad el Concejo Municipal de Osorno resolvió aprobar 20 millones de pesos para desarrollar el programa municipal “Turismo Familiar 2020”, dineros que serán remitidos a la Unión Comunal de Juntas de Vecinos, quienes tendrán la misión de recibir las solicitudes de todas las organizaciones que deseen postular a esta iniciativa vacacional durante la temporada estival. La iniciativa municipal busca brindar espacios de recreación y esparcimiento para las familias de la comuna que, por diversos motivos, no pueden salir de Osorno en esta época. La idea es que tanto niños, adultos y adultos mayores puedan disfrutar tanto en zonas campestres como en balnearios de la zona. La presidenta de la Unión Comunal de Juntas de Vecinos de Osorno, Eliana Catrilef, agradeció la voluntad de la casa edilicia local “al brindar un aporte que permite que las familias puedan optar a planes de veraneo que, de forma particular, les sería difícil o imposible acceder, permitiendo con esto que tanto los ...",2020-01-03,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,21855983.0,chile,radiosago,https://www.radiosago.cl/el-intendente-de-los-lagos-aseguro-que-en-los-proximos-tres-anos-se-entregaran-tres-mil-soluciones-habitacionales/,El Intendente de Los Lagos aseguró que en los próximos tres años se entregarán tres mil soluciones habitacionales,"Es conocido el déficit habitacional que existe en la ciudad de Osorno, y se ha evidenciado en demasía durante el último año con una serie de manifestaciones en torno a esta situación, sin ir más lejos las tomas de campamentos en diferentes zonas de la comuna han demostrado que esto ocurre. El jefe regional Harry Jurgensen, manifestó que se está analizando la creación de un plan especial para entregar tres mil soluciones habitacionales para los próximos tres años en Osorno. En este sentido se espera la visita del Ministro de Vivienda y Urbanismo el 2020 para reafirmar este anuncio. Por su parte, el diputado por Osorno, Fidel Espinoza, indicó que debe haber una suerte de cautela en estos anuncios, ya que se han hecho en el pasado y no se han cumplido a cabalidad. Asimismo dijo que estará de cerca fiscalizando este compromiso, ya que presidirá la comisión de vivienda en la Cámara de Diputados a partir de marzo de 2020. Por parte del Gobierno, el int

## 2. Extraer los topicos utilizando LDA

In [462]:
noticias = corpus.text.values.tolist()

In [463]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

In [464]:
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [465]:
##Preprocesamiento
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

In [466]:
def text_to_list(noticia):
    list_of_words = []
    
    try:
        doc = nlp(noticia)

        for token in doc:
            if (token.pos_=="NOUN"):
                list_of_words.append(token.text)

        for ent in doc.ents:
            if (ent.label_ == "PER" and " " in ent.text):
                list_of_words.append(ent.text)

        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            list_of_words.append(span.text)
    
    except Exception as e: 
        print(noticia)
        print(e)
    
    return list_of_words

In [467]:
noticias_procesadas = []

for index, noticia in enumerate(noticias):
    print(index)
    noticia_procesada = text_to_list(noticia)
    noticias_procesadas.append(noticia_procesada)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [468]:
#Procesamiento con LDA

id2word = corpora.Dictionary(noticias_procesadas)
id2word

In [469]:
texts = noticias_procesadas
dataset = [id2word.doc2bow(noticia_procesada) for noticia_procesada in noticias_procesadas]

In [470]:
print(dataset[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 3), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 3), (74, 1), (75, 1), (76, 3), (77, 1), (78, 2), (79, 2), (80, 2), (81, 1), (82, 1)], [(18, 1), (30, 1), (43, 1), (50, 1), (73, 1), (81, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 3), (1

### Entrenamiento modelo

#### Encontrar el número óptimo de tópicos


In [471]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        
        model = gensim.models.ldamodel.LdaModel(corpus=dataset,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [472]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=dataset, texts=noticias_procesadas, start=2, limit=15, step=2)

2
4


C:\Users\Gerso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
C:\Users\Gerso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


6
8
10
12
14


In [473]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.2711
Num Topics = 4  has Coherence Value of nan
Num Topics = 6  has Coherence Value of 0.31
Num Topics = 8  has Coherence Value of nan
Num Topics = 10  has Coherence Value of nan
Num Topics = 12  has Coherence Value of nan
Num Topics = 14  has Coherence Value of 0.4537


#### NOTA:
Modelo Optimo = 6

In [474]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=dataset,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [475]:
lda_model.save('./output/Tarea_1_LosLagos')

In [476]:
from pprint import pprint

pprint(lda_model.print_topics(num_words=100))

[(0,
  '0.027*"horas" + 0.011*"año" + 0.008*"aumento" + 0.006*"comuna" + '
  '0.006*"meses" + 0.006*"conciertos" + 0.006*"recital" + 0.006*"vez" + '
  '0.005*"octubre" + 0.005*"ciudad" + 0.005*"enero" + 0.005*"dúo" + '
  '0.004*"región" + 0.004*"martes" + 0.004*"puntos porcentuales" + '
  '0.004*"jornada" + 0.004*"cargo" + 0.003*"concierto" + 0.003*"domingo" + '
  '0.003*"trimestre" + 0.003*"parte" + 0.003*"febrero" + 0.003*"cifra" + '
  '0.003*"jueves" + 0.003*"trimestre móvil" + 0.003*"regiones" + '
  '0.003*"iglesia" + 0.003*"trabajo" + 0.003*"comunas" + 0.003*"encuesta" + '
  '0.003*"crisis" + 0.003*"disminución" + 0.003*"piano" + 0.003*"partir" + '
  '0.003*"estallido" + 0.003*"recursos" + 0.002*"año pasado" + 0.002*"años" + '
  '0.002*"septiembre" + 0.002*"presentación" + 0.002*"miércoles" + '
  '0.002*"descentralización" + 0.002*"estallido social" + 0.002*"turistas" + '
  '0.002*"uso" + 0.002*"dependencias" + 0.002*"presentaciones" + 0.002*"poder" '
  '+ 0.002*"alza" + 0.002*"vi

In [477]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, dataset, id2word)
vis

C:\Users\Gerso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.070697 -0.172337       1        1  23.299939
2      0.235801  0.054507       2        1  21.329848
3      0.041354 -0.133893       3        1  15.821070
1     -0.116505  0.118637       4        1  14.384507
0     -0.016734  0.071995       5        1  12.859289
5     -0.073219  0.061091       6        1  12.305346, topic_info=             Term        Freq       Total Category  logprob  loglift
313         horas  269.000000  269.000000  Default  30.0000  30.0000
3365       basura   62.000000   62.000000  Default  29.0000  29.0000
55       proyecto   90.000000   90.000000  Default  28.0000  28.0000
262         salud  108.000000  108.000000  Default  27.0000  27.0000
346        prueba   92.000000   92.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
403      respecto   11.760326   49.134408   Topic6  -6.1282   0.6653
162         parte   12.974301  150.413138   Topic6  -6.0299  -0.3553
1018  condiciones   11.681907   61.290151   Topic6  -6.1349   0.4376
23           días   11.861522   81.607314   Topic6  -6.1196   0.1665
77            vez   11.793974   99.020133   Topic6  -6.1253  -0.0326

[379 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2049      1  0.032036     2019
2049      2  0.224252     2019
2049      3  0.032036     2019
2049      4  0.608684     2019
2049      6  0.096108     2019
...     ...       ...      ...
134       6  0.010829     zona
5074      2  0.991468  árboles
2565      5  0.215190    éxito
2565      6  0.806961    éxito
9031      4  0.955832  órganos

[634 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 2, 1, 6])

In [478]:
def format_topics_documents(ldamodel=None, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_documents(ldamodel=lda_model, corpus=dataset, texts=noticias)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\2222007393.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\Gerso\AppData\Local\Temp\ipykernel_18332\2222007393.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [479]:
#Crea una lista asignandole la comuna a la que se referencia en la noticia (Por defecto se agrega la última mencionada) Así mismo, las que no tienen ninguna comuna
#mensionada se les agrega "Sin especificar"

Lista_Comunas = []
for k in df_dominant_topic['Text']:
    contiene = False
    if(pd.isnull(k)):
        Lista_Comunas.append('Sin especificar')
    else: 
        for z in comunas:
            if(k.__contains__(z)):
                contiene = True
                Lista_Comunas.append(z)
                break
        if(contiene==False):
            Lista_Comunas.append('Sin especificar')
print(len(Lista_Comunas))
for k in Lista_Comunas:
    print(k)
#Creamos un PD con esto:
comuna_Los_Lagos = pd.DataFrame(Lista_Comunas,columns=['comuna'])
comuna_Los_Lagos

422
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Osorno
Puerto Montt
Puerto Montt
Puerto Montt
Puerto Montt
Puerto Montt
Puerto Montt
Puerto Montt
Puerto Montt
Sin especificar
Puerto Montt
Sin especificar
Sin especificar
Puerto Montt
Puerto Montt
Sin especificar
Sin especificar
Sin especificar
Quellón
Sin especificar
Sin especificar
Sin especificar
Puerto Montt
Sin especificar
Sin especificar
Sin especificar
Sin especificar
Puerto Montt
Sin especificar
Sin especificar
Sin especificar
Sin especificar
Sin especificar
Puerto Montt
Llanquihue
Sin especificar
Puerto Montt
Sin especificar
Sin especificar
Sin especificar
Puerto Montt
Sin especificar
Quellón
Sin especificar
Sin especificar
Puerto Montt
Sin especificar
Puerto Montt
Frutillar
Puerto Montt
Sin especificar
Puyehue
Puerto Montt
Osorno
Sin especificar
Sin especificar
Chonchi
Sin especificar
Osorno
Osorno
Osorno
Osorno
Osorno
Llanquihue
Osorno
Osorno
Osorno
Sin especificar
S

,comuna
0,Osorno
1,Osorno
2,Osorno
3,Osorno
4,Osorno
...,...
417,Ancud
418,Castro
419,Sin especificar
420,Ancud


In [480]:
df_dominant_topic=df_dominant_topic.join(comuna_Los_Lagos)  


In [481]:
from pandasql import sqldf 

## 3. Identificar, por cada comuna, cuales son los topicos más importantes

### Está agrupado por comunas, donde muestra el topico dominante y su puntaje en cada comuna

In [490]:

result = sqldf("SELECT * FROM df_dominant_topic GROUP BY comuna ORDER BY topic_Perc_Contrib DESC LIMIT 30")
#Limit 30 porque esa es la cantidad de comunas en Los Lagos
result

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,comuna
0,16,5,0.9932,"investigación, pruebas, fiscal, delitos, años, ruta, víctima, lugar, audiencia, personas","Los hechos ocurrieron entre agosto y septiembre de 2016 y fueron develados por las víctimas en 2017. Las víctimas tenían 10 años de edad al momento en que su padre las agredió sexualmente y en reiteradas ocasiones. De acuerdo a la sentencia, los hechos acreditados con las pruebas presentadas por la Fiscalía Local de Puerto Montt y que incluyó el relato de ambas víctimas, ocurrieron, cuando el acusado se encontraba al cuidado de las menores mientras la madre salía a trabajar. El hombre, en esa época se encontraba con beneficio carcelario a raíz de una pena por un delito de robo. La fiscal que llevó la causa a juicio oral, Ana María Agüero, indicó que el hombre fue condenado como autor del delito consumado de violación impropia en carácter de reiterado. En la sentencia, los jueces Francisco del Campo, Jaime Rojas y Andrés Villagra indicaron que tuvieron en consideración medios de prueba aportados por la Fiscalía y en especial las declaraciones concordantes de las víctimas. La fiscal...",Puerto Montt
1,86,4,0.9829,"personas, prueba, años, enero, horas, estudiantes, lugar, mañana, año, proceso","Durante la jornada de este viernes un motociclista murió tras impactar contra un taxi colectivo en la ciudad de Castro. Según informó La Estrella de Chiloé, los hechos ocurrieron alrededor de las 17:00 en calle José María Caro de la población Raúl Silva Henríquez cuando el motorista, identificado como Daniel Alberto Miranda Nieto, de 23 años, impactó contra un colectivo que viajaba en sentido contrario, salió eyectado de su moto, y se golpeó la cabeza contra el pavimento. Posteriormente el joven fue trasladado al Hospital Augusto Riffart de Castro, donde falleció producto de las heridas a eso de las 20:00. Según determinó el SIAT de Carabineros, el joven viajaba “sin elementos de seguridad”. Mientras que el chófer del colectivo tenía su documentación al día y sin signos de haber ingerido alcohol.",Castro
2,87,2,0.9695,"comuna, actividad, niños, años, parque, programa, año, lugar, zona, gente","Desde hace algún tiempo, ha crecido el interés en la comunidad local por la práctica del hockey y es así como el próximo 18 de enero se realizará una Clínica de Hockey Césped, organizada por el club de hockey Tribu Austral, a cargo del seleccionado Nacional Pablo Purcell. La actividad se llevará a cabo en el Colegio Puerto Varas, en dos horarios: niños de 4 a 17 años de 10 a 12 y adultos de 14 a 16 horas. Valores: Niños: $10.000 y Adultos: $15.000.",Puerto Varas
3,71,4,0.9401,"personas, prueba, años, enero, horas, estudiantes, lugar, mañana, año, proceso","Un incendio consumió la vivienda de una ama de casa en la comuna de Chonchi, quien afortunadamente fue auxiliada por sus vecinos. Los hechos se dieron a eso de las 10:15 de esta mañana en la localidad de Chanquín (en lago Huelde), cuando inició un incendio que consumió por completo la vivienda de 70 metros cuadrados. La única moradora en ese minuto había sido operada recientemente y se encontraba imposibilitada de salir por sus propios medios, pero sus vecinos la ayudaron a escapar por la ventana. Al lugar concurrieron cerca de 20 voluntarios de Bomberos de Cucao, Huillinco y Chonchi para controlar las llamas. Miguel Sánchez, segundo comandante de Bomberos, explicó a Soy Chile que el trabajo de los voluntarios evitó que el fuego se propagara a viviendas contigua, y destacó la acción heroica de los vecinos que ayudaron a una mujer. En total tres personas quedaron damnificadas con la tragedia.",Chonchi
4,232,4,0.9181,"personas, prueba, años, enero, horas, estudiantes, lugar, mañana, año, proceso","La Oficina Móvil de la Junta Nacional de Auxilio Escolar y Becas (Junaeb), está visitando distintas comunas de la Región de Los Lagos, en el contexto de la postulación y renovación de alumnos beneficiados con la Beca Junaeb. ""La idea e